In [1]:
!pip install transformers datasets evaluate accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (1

### Загружаем и создаем датасет

In [2]:
import pandas as pd


df = pd.read_csv("dataset.csv")

In [3]:
df.head(1)

,datetime,headline,description,content,close,open,high,low
0,2024-04-26,Индия вновь принимает нефть из России на танке...,"Владимир Тихонов, принадлежащий Совкомфлоту, н...","Нефтеналивной танкер, который принадлежит нахо...",89.5,89.21,89.83,88.8


In [4]:
texts = []
labels = []
EPS = 0.05
zero_cnt = 0
labels_dict = {0: [], 1: [], 2:[]}


for ind in df.index:
    text = " ".join([df["headline"][ind], str(df["description"][ind]), str(df["content"][ind])])

    delta = float(df["close"][ind]) - float(df["open"][ind])
    label = 0
    if abs(delta) > EPS:
        if delta > 0:
            label = 1
        else:
            label = 2
    else:
        zero_cnt += 1

    labels_dict[label].append(text)
    texts.append(text)
    labels.append(label)

print("Процентов случаев, когда цена существенно не менялась (0): %.2f" % (zero_cnt / len(df.index) * 100))
for i in range(3):
    print("%d : %d" % (i, len(labels_dict[i])))

Процентов случаев, когда цена существенно не менялась (0): 5.30
0 : 495
1 : 4688
2 : 4156


In [5]:
test_idx = 5

print(texts[test_idx])

Нефть дорожает на данных о снижении запасов сырой нефти в США По данным EIA за неделю, завершившуюся 19 апреля 2024 г., коммерческие запасы нефти в США упали на 6,368 млн барр., до 453,625 млн барр. после роста на 2,735 млн барр. за предыдущую неделю. Цены на нефть растут.

24 апреля 2024 г. стоимость фьючерсов на нефть сорта Brent на июнь на лондонской бирже ICE Futures упала на 0,45%, до 88,02 долл. США/барр.
Фьючерсы на нефть сорта WTI на июнь на электронных торгах Нью-йоркской товарной биржи (NYMEX) подешевели на 0,66%, до 82,81 долл. США/барр.

 25 апреля нефть дорожает.
 К 13:30 мск стоимость июньских фьючерсов на Brent на электронных торгах ICE Futures выросла на 0,06%, до 88,07 долл. США/барр.
 На электронных торгах NYMEX фьючерсы на нефть WTI на июнь подорожали на 0,05%, до 82,85 долл. США/барр.

 Котировки природного газа в США упали до 1,9 долл. США/млн БТЕ (68,02 долл. США/1000 м3).
 24 апреля 2024 г. фьючерсы на природный газ на июнь на электронных торгах NYMEX подешевели 

In [6]:
#!pip install datasets
from datasets import Dataset

dataset = Dataset.from_dict({"text": texts, "label": labels})
dataset = dataset.train_test_split(test_size=0.3)

In [7]:
normalization_enabled = True
if normalization_enabled:
    sum = []
    sum_labels = []
    for i in range(3):
        labels_dict[i] = labels_dict[i][:len(labels_dict[0])]
        sum = sum + labels_dict[i]
        sum_labels = sum_labels + [i] * len(labels_dict[0])

    norm_dataset = Dataset.from_dict({"text": sum, "label": sum_labels})
    norm_dataset = norm_dataset.train_test_split(test_size=0.3)

    print("After and before normalization")
    for d in [norm_dataset, dataset]:
        for s in ["train", "test"]:
            print()
            print("For "+s+" dataset")
            for i in range(3):
                print(str(i) + " : " + str(d[s]["label"].count(i)))
        print("-------------------")


After and before normalization

For train dataset
0 : 366
1 : 336
2 : 337

For test dataset
0 : 129
1 : 159
2 : 158
-------------------

For train dataset
0 : 337
1 : 3283
2 : 2917

For test dataset
0 : 158
1 : 1405
2 : 1239
-------------------


### Бейзлайн

Полный датасет, в котором в пр

In [8]:
from transformers import AutoTokenizer


tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=256)

tokenized_dataset_train = dataset["train"].map(tokenize_function, batched=True)
tokenized_dataset_test = dataset["test"].map(tokenize_function, batched=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/241k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/468k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Map:   0%|          | 0/6537 [00:00<?, ? examples/s]

Map:   0%|          | 0/2802 [00:00<?, ? examples/s]

In [9]:
s = set()
for tmp in tokenized_dataset_train:
    s.add(len(tmp["input_ids"]))

print(s)

{256}


In [10]:
from transformers import AutoModelForSequenceClassification


model = AutoModelForSequenceClassification.from_pretrained("cointegrated/rubert-tiny", num_labels=3)
model.cuda()

model.safetensors:   0%|          | 0.00/47.7M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(29564, 312, padding_idx=0)
      (position_embeddings): Embedding(512, 312)
      (token_type_embeddings): Embedding(2, 312)
      (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-2): 3 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=312, out_features=312, bias=True)
              (key): Linear(in_features=312, out_features=312, bias=True)
              (value): Linear(in_features=312, out_features=312, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=312, out_features=312, bias=True)
              (LayerNorm): LayerNorm((312,), eps=1e-1

In [11]:
import numpy as np
#!pip install evaluate
#!pip install transformers --upgrade
#!pip install transformers[torch]
import evaluate
from transformers import TrainingArguments, Trainer


metric = evaluate.load("accuracy")


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

training_args = TrainingArguments(output_dir="test_trainer",
                                  evaluation_strategy="epoch",
                                  save_strategy="no",
                                  num_train_epochs=3)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [12]:
small_train_dataset = tokenized_dataset_train
small_eval_dataset = tokenized_dataset_test

In [13]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [14]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.871200,0.879383,0.496431
2,0.864200,0.874804,0.486438
3,0.846800,0.890228,0.483940


TrainOutput(global_step=2454, training_loss=0.8526636821619359, metrics={'train_runtime': 85.0662, 'train_samples_per_second': 230.538, 'train_steps_per_second': 28.848, 'total_flos': 72317214844416.0, 'train_loss': 0.8526636821619359, 'epoch': 3.0})

### Нормализованный датасет

In [15]:
if normalization_enabled:
    tokenized_dataset_train = norm_dataset["train"].map(tokenize_function, batched=True)
    tokenized_dataset_test = norm_dataset["test"].map(tokenize_function, batched=True)

Map:   0%|          | 0/1039 [00:00<?, ? examples/s]

Map:   0%|          | 0/446 [00:00<?, ? examples/s]

In [16]:
if normalization_enabled:
    small_train_dataset = tokenized_dataset_train
    small_eval_dataset = tokenized_dataset_test

    training_args = TrainingArguments(output_dir="test_trainer",
                                  evaluation_strategy="epoch",
                                  save_strategy="no",
                                  num_train_epochs=3)

    trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
    )

    trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.903004,0.547085
2,No log,0.732079,0.663677
3,No log,0.718362,0.645740


In [17]:
if normalization_enabled:
    tokenized_dataset_train = dataset["train"].map(tokenize_function, batched=True)
    tokenized_dataset_test = dataset["test"].map(tokenize_function, batched=True)

Map:   0%|          | 0/6537 [00:00<?, ? examples/s]

Map:   0%|          | 0/2802 [00:00<?, ? examples/s]

In [18]:
if normalization_enabled:
    small_train_dataset = tokenized_dataset_train
    small_eval_dataset = tokenized_dataset_test

    training_args = TrainingArguments(output_dir="test_trainer",
                                  evaluation_strategy="epoch",
                                  save_strategy="no",
                                  num_train_epochs=3)

    trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
    )

    trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.814900,0.966192,0.475375
2,0.673200,1.070115,0.488580
3,0.609900,1.191003,0.484654


### Большая модель

In [19]:
model = AutoModelForSequenceClassification.from_pretrained("ai-forever/sbert_large_nlu_ru", num_labels=3)
model.cuda()

config.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ai-forever/sbert_large_nlu_ru and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(120138, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-23): 24 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((

In [20]:
tokenizer = AutoTokenizer.from_pretrained("ai-forever/sbert_large_nlu_ru")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=256)

small_train_dataset = dataset["train"].map(tokenize_function, batched=True)
small_eval_dataset = dataset["test"].map(tokenize_function, batched=True)

tokenizer_config.json:   0%|          | 0.00/323 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.78M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Map:   0%|          | 0/6537 [00:00<?, ? examples/s]

Map:   0%|          | 0/2802 [00:00<?, ? examples/s]

In [21]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.630200,1.154124,0.484297
2,0.500700,1.502892,0.490007
3,0.395300,1.722148,0.489293


TrainOutput(global_step=2454, training_loss=0.5065744426450597, metrics={'train_runtime': 99.4451, 'train_samples_per_second': 197.204, 'train_steps_per_second': 24.677, 'total_flos': 72317214844416.0, 'train_loss': 0.5065744426450597, 'epoch': 3.0})